In [2]:
# %env CUDA_VISIBLE_DEVICES=1
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
# %env TF_GPU_ALLOCATOR=cuda_malloc_async
# %env XLA_PYTHON_CLIENT_MEM_FRACTION=.99
# %env JAX_PLATFORMS=cpu

from robotodo.algos.pi0.nn.pi0 import Pi0
import jax
import flax.nnx

# mesh = jax.sharding.Mesh(jax.devices(), ("x",))
# sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec())

pi0 = Pi0(flax.nnx.Rngs(0))

key = jax.random.key(0)
observation = pi0.observation_spec.random()
action = pi0.action_spec.random()
timestep = jax.numpy.asarray([0])

env: XLA_PYTHON_CLIENT_PREALLOCATE=false


Could not load symbol cuFuncGetName. Error: /usr/lib/x86_64-linux-gnu/libcuda.so.1: undefined symbol: cuFuncGetName


In [7]:
loss = pi0.compute_loss(
    key,
    observation,
    action,
)
loss.block_until_ready()

Array([[2.7631567]], dtype=float32)

In [7]:
%%timeit -n 10

loss = pi0.compute_loss(
    key,
    pi0.observation_spec.random(),
    pi0.action_spec.random(),
)
loss.block_until_ready()


47.4 ms ± 4.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%%timeit -n 1

# %%prun
loss = pi0.compute_loss(
    key,
    observation,
    action,
)
loss.block_until_ready()

43.9 ms ± 632 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit -n 1
pi0._embed_prefix(
    observation,
)

2.03 s ± 240 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit -n 1

# without nnx.jit
computed_actions = pi0.sample_actions(
    key,
    observation,
)
computed_actions["numeric_states"].block_until_ready()

6.91 s ± 2.46 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit -n 1

# with nnx.jit 
computed_actions = pi0.sample_actions(
    key,
    observation,
)
computed_actions["numeric_states"].block_until_ready()

71.6 ms ± 6.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
